In [1]:
%pip install --upgrade -q pip setuptools wheel
%pip install -q Pylance PyObjC pythainlp openai gtts matplotlib playsound icrawler fastapi

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [10]:
import matplotlib as mpl
import os
import matplotlib.font_manager
if os.path.exists('./Sarabun-Regular.ttf') == False:
    os.system('wget -q https://github.com/google/fonts/raw/main/ofl/sarabun/Sarabun-Regular.ttf')

mpl.font_manager.fontManager.addfont('Sarabun-Regular.ttf')
mpl.rc('font', family='Sarabun')

In [2]:
import logging
logging.disable(logging.CRITICAL)

In [6]:
import os
import shutil
import random
import matplotlib.pyplot as plt

from PIL import Image
from gtts import gTTS
from openai import OpenAI
from pythainlp.tokenize import word_tokenize
from icrawler.builtin import GoogleImageCrawler

def create_opentyphoon_client():
    API_KEY='sk-psIjnhS63MFqDJzmrCnY6iLPr3iZ6wAXP57La0dvUcNoE9dc'
    return OpenAI(
        api_key=API_KEY,
        base_url="https://api.opentyphoon.ai/v1",
    )

def get_random_file_path(directory):
    all_items = os.listdir(directory)
    files = [f for f in all_items if os.path.isfile(os.path.join(directory, f))]
    if not files:
        return None
    random_file = random.choice(files)
    return os.path.join(directory, random_file)

def cue_hierachy(state, target_word, client):
    semantic_cue = f"""คุณเป็นนักบำบัดการพูดที่กำลังช่วยผู้ป่วยที่มีภาวะเสียการพูดฝึกฝนทักษะการพูดและการสื่อสาร จากโรค Aphasia
    ผู้ป่วยอาจมีปัญหาในการหาคำและการสร้างประโยค คุณมีหน้าที่ช่วยผู้ป่วยให้สามารถคิดคำที่คุณใบ้ออก <โดยห้ามบอกถึงโดยตรง>
    การใบ้คนไข้คุณต้องอธิบายประเภทของสิ่งที่จะใบ้ เช่น
    ตัวอย่างการใบ้คุณยประเภทสำหรับคำว่า "แอปเปิ้ล"
    มันเป็นผลไม้
    ตัวอย่างการใบ้คุณยประเภทสำหรับคำว่า "รถยนต์"
    มันคือยานพาหนะ
    """

    sentence_completion_cue = f"""คุณเป็นนักบำบัดการพูดที่กำลังช่วยผู้ป่วยที่มีภาวะเสียการพูดฝึกฝนทักษะการพูดและการสื่อสาร จากโรค Aphasia
    ผู้ป่วยอาจมีปัญหาในการหาคำและการสร้างประโยค คุณมีหน้าที่ช่วยผู้ป่วยให้สามารถคิดคำที่คุณใบ้ออก <โดยห้ามบอกถึงโดยตรง>
    การใบ้คนไข้คุณต้องสร้างประโยคที่กำลังใช้หรือมีปฏิสัมพันธ์ของสิ่งที่จะใบ้ เช่น
    ตัวอย่างการใบ้คำว่า "แอปเปิ้ล"
    ผมกำลังเดินทางไปตลาดเผื่อซื้อผลไม้สีแดงสดมาทำขนมพาย
    """

    phonemic_cue = f"""คุณเป็นนักบำบัดการพูดที่กำลังช่วยผู้ป่วยที่มีภาวะเสียการพูดฝึกฝนทักษะการพูดและการสื่อสาร จากโรค Aphasia
    ผู้ป่วยอาจมีปัญหาในการหาคำและการสร้างประโยค คุณมีหน้าที่ช่วยผู้ป่วยให้สามารถคิดคำที่คุณใบ้ออก <โดยห้ามบอกถึงโดยตรง>
    การใบ้คนไข้คุณต้องอธิบายลักษณะของเสียงพยัญชนะแรกของสิ่งที่จะใบ้ของคำๆนั้น เช่น
    ตัวอย่างการใบ้คำว่า "แอปเปิ้ล"
    เสียง ออ
    ตัวอย่างการใบ้คำว่า "กล้วย"
    เสียง กอ
    ตัวอย่างการใบ้คำว่า "คน"
    เสียง คอ
    """

    functional_description_cue = f"""คุณเป็นนักบำบัดการพูดที่กำลังช่วยผู้ป่วยที่มีภาวะเสียการพูดฝึกฝนทักษะการพูดและการสื่อสาร จากโรค Aphasia
    ผู้ป่วยอาจมีปัญหาในการหาคำและการสร้างประโยค คุณมีหน้าที่ช่วยผู้ป่วยให้สามารถคิดคำที่คุณใบ้ออก <โดยห้ามบอกถึงโดยตรง>
    การใบ้คนไข้คุณต้องอธิบายลักษณะของสิ่งที่จะใบ้ เช่น
    ตัวอย่างการใบ้คุณลักษณะสำหรับคำว่า "แอปเปิ้ล"
    มันเป็นผลไม้ที่เอาไว้กิน มีสีแดงหรือสีเขียว มีผิวที่กลมและเรียบ มีขายตามห้างหรือร้านขายผลไม้
    """
    
    if state == 0:
        mode = semantic_cue
    elif state == 1:
        mode = sentence_completion_cue
    elif state == 2:
        mode = phonemic_cue
    elif state == 4:
        mode = functional_description_cue
    else:
        return None

    stream = client.chat.completions.create(
    model="typhoon-instruct",
    messages=[
        {
            "role": "systemp",
            "content": f'{mode}',
        },
        {
            "role": "user",
            "content": f"ผมอยากพูดคำว่า {target_word}",
        }
    ],
    
    max_tokens=300,
    temperature=0,
    top_p=0.99,
    stream=True,
    )

    respond=[]
    for chunk in stream:
        if hasattr(chunk, 'choices') and len(chunk.choices) > 0:
            choice = chunk.choices[0]
            if hasattr(choice, 'delta') and hasattr(choice.delta, 'content'):
                if choice.delta.content is not None:
                    respond.append(choice.delta.content)

    return "".join(respond).replace(target_word, '')

0.Semantic Cue : อธิบายประเภท

1.Sentence Completion Cue :สร้างประโยคที่เกี่ยวข้อง

2.Phonemic Cue : อธิบายเสียงแรก

3.Modeling Cue : เล่นเสียง

4.Functional Description Cue : อธิบายรายละเอียด

5.Printed Word Cue : เปิดคำ

In [7]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn
import nest_asyncio
import json

app = FastAPI()

client = create_opentyphoon_client()

class Cue_HierarchyRequest(BaseModel):
    state: int
    target_word: str
    
class Check_User_InputRequest(BaseModel):
    state: int
    target_word: str
    user_input: str
  
def call_cue_hierarchy(state, target_word, client=client):
    respond=None
    audio_path=None
    img_path=None
    if state >= 0 and state <= 2:
        respond = cue_hierachy(state, target_word, client)
    elif state == 3:
        tts = gTTS(target_word, lang='th')
        tts.save('temp.mp3')
        audio_path='./temp.mp3'
    elif state == 4:
        if os.path.exists('./img'):
            shutil.rmtree('./img')
            
        google_crawler = GoogleImageCrawler(storage={'root_dir': './img'})
        google_crawler.crawl(keyword=target_word, max_num=3)
        img_path=get_random_file_path('./img')
    elif state == 5:
        tts = gTTS(target_word, lang='th')
        tts.save('temp.mp3')
        audio_path='./temp.mp3'
        plt.figure(figsize=(4, 2))
        plt.text(0.5, 0.5, target_word, fontsize=36, ha='center', va='center', transform=plt.gca().transAxes)
        plt.axis('off')
        img_path='./target_word.jpg'
        plt.savefig('./target_word.jpg')
    return_format={"state" : state}
    if respond != None:
        return_format = {"respond" : respond,}
    if audio_path != None:
        return_format.update({"audio_path": audio_path})
    if img_path != None:
        return_format.update({"img_path": img_path})
    return return_format
        
def check_user_input(target_word, user_input):
    user_respond = word_tokenize(user_input, engine="newmm")
    if target_word in user_respond:
        return True
    else:
        return False

In [8]:
call_cue_hierarchy(0, 'เป็ด')

{'respond': 'มันเป็นสัตว์ชนิดหนึ่ง ที่มีขนาดเล็ก และมีปีก'}

In [10]:
@app.post("/gen_question")
async def cue_hierarchy_endpoint(request: Cue_HierarchyRequest):
        result = call_cue_hierarchy(request.state, request.target_word)
        return result
    
@app.post("/check_answer")
async def check_user_input_endpoint(request: Check_User_InputRequest):
        result = check_user_input(request.target_word, request.user_input)
        return result
    

nest_asyncio.apply()
uvicorn.run(app, port=12000)

INFO:     Started server process [10679]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


INFO:     127.0.0.1:53232 - "POST /gen_question HTTP/1.1" 200 OK
INFO:     127.0.0.1:53232 - "POST /gen_question HTTP/1.1" 200 OK
